BDLE 2022

date du document  :  16/12/2022

# TP 9 et 10 : traitement sur des partitions : tri, regroupement, fenetre, jointure
Version ETUDIANT



## Préparation

Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/essai

In [68]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/essai"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [69]:
!pip install -q pyspark
!pip install -q findspark
print("installé")

installé


Démarrer la session spark

In [70]:
import os
# !find /usr/local/lib/ -name "pyspark" 
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.8/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [71]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","3G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1672606921303


Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [72]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [73]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select') or r.lower().startswith('with') )
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

Utiliaires : Chronomètres

In [74]:
#------------------------------
# Chronometre : chronoPersist2
#------------------------------
import time

# Ce chronometre garantit que chaque tuple du dataframe est lu entièrement.
# En effet il est nécessaire de lire le détail de chaque tuple avant de les 'copier' en mémoire.
def chronoPersist(df):
    df.unpersist()
    t1 = time.perf_counter()
    count = df.persist().count()
    t2 = time.perf_counter()
    df.unpersist()
    print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')

def chronoPersist2(df):
  dest = df.selectExpr("*", "1")
  t1 = time.perf_counter()
  count = dest.persist().count()
  t2 = time.perf_counter()
  dest.unpersist()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')
        
def chronoCount(df):
  t1 = time.perf_counter()
  count = df.count()
  t2 = time.perf_counter()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour dénombrer', count , 'elements')
    
print("fonctions définies")

fonctions définies


## Accès aux données

In [75]:
import os
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

['ratings3M.zip',
 'movies.csv',
 'notes1M.zip',
 'notes1M.json',
 'films.json',
 'ratings3M.csv']

URL pour l'accès aux datasets

In [76]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [77]:
import os
from urllib import request
import zipfile

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

def unzip_file(local_dir, file):
  with zipfile.ZipFile(local_dir + "/" + file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)
  # os.remove(local_dir + "/" + file)


web_dir = PUBLIC_DATASET + "/movielens"

download_file(web_dir, local_dir, "notes1M.zip")
unzip_file(local_dir, "notes1M.zip")

download_file(web_dir, local_dir, "ratings3M.zip")
unzip_file(local_dir, "ratings3M.zip")

download_file(web_dir, local_dir, "films.json")

web_dir = PUBLIC_DATASET + "/movielens/ml-latest"

download_file(web_dir, local_dir, "movies.csv")


# Liste des fichiers
os.listdir(local_dir)

notes1M.zip is already stored
ratings3M.zip is already stored
films.json is already stored
movies.csv is already stored


['ratings3M.zip',
 'movies.csv',
 'notes1M.zip',
 'notes1M.json',
 'films.json',
 'ratings3M.csv']

### Dataframe les Films


*   On récupère un échantillion de films
*   On partitionne les données films dans 3 partitions



In [78]:
# schema_film = "nf long, titre String, g Array<String>"
schema_film = StructType([StructField('nF',LongType()),
                          StructField('titre',StringType()),
                          StructField('g',ArrayType(StringType()))])

films = spark.read.json(local_dir + "/" + "films.json", schema = schema_film).selectExpr("nF", "titre", "g as genres")
# print('schema:', films.schema)

# extrait
film_extrait = films.where("nF <100").repartition(3).persist()

film_extrait.printSchema()
display(film_extrait)

root
 |-- nF: long (nullable = true)
 |-- titre: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)



,nF,titre,genres
0,94,Beautiful Girls (1996),"[Comedy, Drama, Romance]"
1,45,To Die For (1995),"[Comedy, Drama, Thriller]"
2,58,"Postman, The (Postino, Il) (1994)","[Comedy, Drama, Romance]"
3,54,"Big Green, The (1995)","[Children, Comedy]"
4,9,Sudden Death (1995),[Action]
...,...,...,...
87,38,It Takes Two (1995),"[Children, Comedy]"
88,15,Cutthroat Island (1995),"[Action, Adventure, Romance]"
89,85,Angels and Insects (1995),"[Drama, Romance]"
90,64,Two if by Sea (1996),"[Comedy, Romance]"


### Dataframe Les Notes



*   On récupère un échantillion de notes, en gardant celles qui référence un film existant
*   On partitionne les données de notes dans 3 partitions



In [79]:
notes_schema = StructType([StructField('nF',LongType()),
                           StructField('nU',LongType()),
                           StructField('note',DoubleType()),
                           StructField('annee',LongType())])

notes = spark.read.json(local_dir + "/" + "notes1M.json", schema = notes_schema).selectExpr("nF", "nU", "note", "annee")

#extrait
notes_extrait = notes.where("nU < 1000").join(film_extrait, "nF").select(notes["nF"], "nU", "note", "annee").repartition(3).persist()
print(notes_extrait.count())
display(notes_extrait)

185


,nF,nU,note,annee
0,19,76,1.5,2011
1,62,691,5.0,1996
2,25,832,5.0,1998
3,5,296,3.5,2005
4,48,781,3.0,1996
...,...,...,...,...
95,69,798,3.0,2000
96,95,430,3.0,1996
97,10,984,4.0,1996
98,36,261,2.0,1997


# Exercice 1 : Traitement sur des partitions

#### Fonction showPartitionSize

La fonction *showPartitionSize*  affiche le nombre d'éléments dans chaque partition

In [80]:
# La fonction part_size prend en paramètres un numéro de partition (partID) et un itérateur sur une partition.
# Elle retourne un itérateur sur une partition qui contient un seul tuple (num_partition, nbtuples).

def partSize(partID, iterateur):
  size=0
  suivant = next(iterateur, None)
  while suivant is not None :
    size += 1
    suivant = next(iterateur, None)
  yield (partID, size)


def showPartitionSize(df):
  if df.isEmpty():
    print("empty dataframe")
  else:
    #invoquer la fonction partSize sur chaque partition 
    t = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).toDF(['partID', 'size'])

    # Rmq : selectExpr("1") sert à simplifier la partition pour ne garder qu'une seule colonne contentant des "1",
    # ce qui est suffisant pour compter le nombre de tuples dans la partition.

    #affichage
    t.show()

print('showPartitionSize définie')

showPartitionSize définie


#### Fonction showPartitions
 
La fonction *showPartitions*  affiche les _n_ premiers éléments de chaque partition


In [81]:
def showPartitions(df, N=5 , display_per_partition=True):
  if df.isEmpty():
     print("empty dataframe")
  else:   
    nb_partitions = df.rdd.getNumPartitions()

    # fonction topN est invoquée sur une partition.
    # topN retourne un itérateur sur une partition qui contient les N premiers éléments de cette partition
    def topN(partID, iterateur):
      c=0
      suivant = next(iterateur, None)
      while suivant is not None and c < N :
        c+=1
        tuple_avec_numero_partition = (partID, *suivant)
        suivant = next(iterateur, None)
        yield tuple_avec_numero_partition
    #-- fin de topN ---

    partid_tmp = "_p_"
    nom_attributs = [partid_tmp] + df.schema.fieldNames()
    premiers_tuples = df.rdd.mapPartitionsWithIndex(topN).toDF(nom_attributs)
    premiers_tuples.persist()

    if(display_per_partition):
      # afficher séparément le contenu du début de chaque partition
      for partID in range(nb_partitions):
        print("partition", partID)
        # afficher la partition partID
        premiers_tuples.where(col(partid_tmp)==partID).drop(partid_tmp).show(N, False)
    else:
      return display(premiers_tuples)
   
print('showPartitions définie')

showPartitions définie


Afficher la taille de chaque partition

In [82]:
showPartitionSize(film_extrait)

+------+----+
|partID|size|
+------+----+
|     0|  31|
|     1|  31|
|     2|  30|
+------+----+



Afficher les premiers éléments de chaque partition

In [83]:
showPartitions(film_extrait,5)

partition 0
+---+---------------------------------+-------------------------+
|nF |titre                            |genres                   |
+---+---------------------------------+-------------------------+
|94 |Beautiful Girls (1996)           |[Comedy, Drama, Romance] |
|45 |To Die For (1995)                |[Comedy, Drama, Thriller]|
|58 |Postman, The (Postino, Il) (1994)|[Comedy, Drama, Romance] |
|54 |Big Green, The (1995)            |[Children, Comedy]       |
|9  |Sudden Death (1995)              |[Action]                 |
+---+---------------------------------+-------------------------+

partition 1
+---+--------------------------------------------+-----------------+
|nF |titre                                       |genres           |
+---+--------------------------------------------+-----------------+
|14 |Nixon (1995)                                |[Drama]          |
|71 |Fair Game (1995)                            |[Action]         |
|49 |When Night Is Falling (1995)   

## Question 1 : Tri des films par titre

En vous inspirant de la fonction *topN*  définir une fonction qui tri les données de *film_extrait* par titre. Le résultat a le meme schéma que film.



*   Dans la table de films, pour chaque partition, on partitionne encore les données en 3 groupes différents.
*   De cette façon , on obtient une table distribuée où chaque table où danc chaque partitions, on a plusieurs groupe de données.



In [84]:

def tri_par_titre(iterateur):

  suivant = next(iterateur, None)
  groupes = [[] for x in range(3)]

  while suivant is not None:
    lettre1 = suivant.titre[0]
    if lettre1 < 'H':
      p=0
    elif lettre1 < 'P':
      p=1
    else:
      p=2
    groupes[p].append((p, *suivant))
    suivant = next(iterateur, None)

  for groupe in groupes:
    for film in sorted(groupe, key=lambda t: t[2]): # tri selon film ( indice 2 dans t)
      yield film

# tester la fonction
# it = iter(film_extrait.limit(3).collect())
# for x in tri_par_titre(it):
#   print(x)

names = ["p"] + film_extrait.schema.fieldNames()
film1 = film_extrait.rdd.mapPartitions(tri_par_titre).toDF(names)
showPartitions(film1,30)

partition 0
+---+---+----------------------------------------------------+-------------------------------------------------+
|p  |nF |titre                                               |genres                                           |
+---+---+----------------------------------------------------+-------------------------------------------------+
|0  |13 |Balto (1995)                                        |[Adventure, Animation, Children]                 |
|0  |94 |Beautiful Girls (1996)                              |[Comedy, Drama, Romance]                         |
|0  |54 |Big Green, The (1995)                               |[Children, Comedy]                               |
|0  |88 |Black Sheep (1996)                                  |[Comedy]                                         |
|0  |35 |Carrington (1995)                                   |[Drama, Romance]                                 |
|0  |39 |Clueless (1995)                                     |[Comedy, Romance]     

repartitionner les données par **intervalle** en fonction de l'attribut p



*   On shuffle les donneés de la table films
*   C'est à dire que dans chaque partitions, on associe les données du même groupe



In [85]:
film2 = film1.repartitionByRange(3, col('p'))
showPartitions(film2,1000)

partition 0
+---+---+-------------------------------------------------------------------------------+--------------------------------------------+
|p  |nF |titre                                                                          |genres                                      |
+---+---+-------------------------------------------------------------------------------+--------------------------------------------+
|0  |13 |Balto (1995)                                                                   |[Adventure, Animation, Children]            |
|0  |94 |Beautiful Girls (1996)                                                         |[Comedy, Drama, Romance]                    |
|0  |54 |Big Green, The (1995)                                                          |[Children, Comedy]                          |
|0  |88 |Black Sheep (1996)                                                             |[Comedy]                                    |
|0  |35 |Carrington (1995)                 



*   Dans la première partition j'ai tous les films dont la première lettre du titre est < H

*   Dans la deuxième partition j'ai tous les films dont la première lettre du titre est entre H et P (non inclus)

*   Dans la première partition j'ai tous les films dont la première lettre du titre est P et Z .






In [86]:
film_tri_total = film2.sortWithinPartitions('titre')
showPartitions(film_tri_total,1000)

partition 0
+---+---+-------------------------------------------------------------------------------+--------------------------------------------+
|p  |nF |titre                                                                          |genres                                      |
+---+---+-------------------------------------------------------------------------------+--------------------------------------------+
|0  |19 |Ace Ventura: When Nature Calls (1995)                                          |[Comedy]                                    |
|0  |37 |Across the Sea of Time (1995)                                                  |[Documentary, IMAX]                         |
|0  |11 |American President, The (1995)                                                 |[Comedy, Drama, Romance]                    |
|0  |85 |Angels and Insects (1995)                                                      |[Drama, Romance]                            |
|0  |82 |Antonia's Line (Antonia) (1995)   

## Question 2 : Regrouper les films par genre et les compter

En vous inspirant de la fonction *topN*  définir une fonction qui compte le nombre de films par genre. Le schéma du résultat est (genre, n)

In [87]:

def count_par_genre(iterateur):

  D=dict()

  suivant = next(iterateur, None)

  while suivant is not None:
    for genre in suivant.genres :
      if genre not in D:
        D[genre] = 1
      else:
        D[genre]+=1

    suivant = next(iterateur, None)

  for genre,nb in D.items():
      yield (genre,nb)


In [88]:
names = ["genre", "n"]
count_genre = film_extrait.rdd.mapPartitions(count_par_genre).toDF(names)

showPartitions(count_genre,5)

partition 0
+--------+---+
|genre   |n  |
+--------+---+
|Comedy  |13 |
|Drama   |19 |
|Romance |10 |
|Thriller|5  |
|Children|6  |
+--------+---+

partition 1
+--------+---+
|genre   |n  |
+--------+---+
|Drama   |17 |
|Action  |8  |
|Romance |7  |
|Children|4  |
|Mystery |2  |
+--------+---+

partition 2
+--------+---+
|genre   |n  |
+--------+---+
|Comedy  |11 |
|Drama   |14 |
|Crime   |4  |
|Thriller|7  |
|Action  |4  |
+--------+---+



## Question 3 : Numérotation

En vous inspirant de la fonction *topN*  définir une fonction qui attribue un numéro de 1 à n à chaque tuple de notes_extrait. Le résultat a les attributs de notes + un attribut num.

In [89]:

def numeroter(iterateur):

  suivant = next(iterateur, None)
  
  c=1
  while suivant is not None:
    yield (*suivant,c)
    suivant = next(iterateur, None)
    c+=1



In [90]:
names = notes_extrait.schema.fieldNames() + ["num"]
notes_extrait_num = notes_extrait.rdd.mapPartitions(numeroter).toDF(names)

showPartitions(notes_extrait_num,5) 

partition 0
+---+---+----+-----+---+
|nF |nU |note|annee|num|
+---+---+----+-----+---+
|19 |76 |1.5 |2011 |1  |
|62 |691|5.0 |1996 |2  |
|25 |832|5.0 |1998 |3  |
|5  |296|3.5 |2005 |4  |
|48 |781|3.0 |1996 |5  |
+---+---+----+-----+---+

partition 1
+---+---+----+-----+---+
|nF |nU |note|annee|num|
+---+---+----+-----+---+
|15 |875|4.0 |1996 |1  |
|62 |368|4.0 |2006 |2  |
|29 |592|4.0 |2000 |3  |
|31 |767|3.0 |1996 |4  |
|25 |339|5.0 |1997 |5  |
+---+---+----+-----+---+

partition 2
+---+---+----+-----+---+
|nF |nU |note|annee|num|
+---+---+----+-----+---+
|19 |151|2.0 |2004 |1  |
|48 |506|4.0 |2010 |2  |
|88 |168|3.0 |1996 |3  |
|26 |366|5.0 |1997 |4  |
|34 |711|2.5 |2009 |5  |
+---+---+----+-----+---+



## Question 4 : Regrouper et trier

Définir une fonction qui affiche le nombre de notes par année dans l'ordre croissant des années

In [91]:

def regrouper_trier(iterateur):

  suivant = next(iterateur, None)
  D=dict()

  while suivant is not None:

    annee = suivant.annee

    if annee not in D:
        D[annee] = 1
    else:
        D[annee]+=1
    suivant = next(iterateur, None)

  for annee in sorted(D.keys()):
      yield (annee,D[annee])



In [92]:
names = ['annee','nb_note']
nb_notes_by_annee = notes_extrait.rdd.mapPartitions(regrouper_trier).toDF(names)

showPartitions(nb_notes_by_annee,5) 

partition 0
+-----+-------+
|annee|nb_note|
+-----+-------+
|1996 |17     |
|1997 |6      |
|1998 |3      |
|1999 |6      |
|2000 |3      |
+-----+-------+

partition 1
+-----+-------+
|annee|nb_note|
+-----+-------+
|1996 |22     |
|1997 |6      |
|1999 |1      |
|2000 |6      |
|2001 |1      |
+-----+-------+

partition 2
+-----+-------+
|annee|nb_note|
+-----+-------+
|1996 |13     |
|1997 |8      |
|1999 |5      |
|2000 |5      |
|2001 |6      |
+-----+-------+



## Question 5 : top fréquence

Définir une fonction qui affiche les 5 mots les plus fréquents dans les titres

In [93]:

def mots_freq(iterateur):

  suivant = next(iterateur, None)
  D=dict()

  while suivant is not None:

    titre = suivant.titre
    words = titre.split(" ")

    for w in words:
      if w not in D:
          D[w] = 1
      else:
          D[w]+=1

    suivant = next(iterateur, None)

  i=0
  for word, freq in sorted(D.items(), key=lambda x: x[1], reverse=True):
    if(i==5):
      break
    yield (word,freq)
    i+=1


In [94]:
names = ['mot','freq']
most_freq_word = film_extrait.rdd.mapPartitions(mots_freq).toDF(names)

showPartitions(most_freq_word) 

partition 0
+------+----+
|mot   |freq|
+------+----+
|(1995)|24  |
|(1996)|6   |
|The   |3   |
|to    |2   |
|White |2   |
+------+----+

partition 1
+------+----+
|mot   |freq|
+------+----+
|(1995)|27  |
|the   |4   |
|When  |3   |
|The   |3   |
|in    |3   |
+------+----+

partition 2
+------+----+
|mot   |freq|
+------+----+
|(1995)|24  |
|(1996)|5   |
|The   |4   |
|and   |3   |
|the   |3   |
+------+----+



## Question 6 : Jointure

Définir les fonctions pour calculer la jointure entre les extraits de films et de notes



*   La jointure se fera entre film_extrait et notes_extrait

*   On définit J : la jointure entre film_extrait et notes_extrait

*   Le but est de proposer une solution pour calculer J , en utilisant un traitement parralèle et distribué des jointures







In [95]:
J = film_extrait.join(notes_extrait, film_extrait.nF == notes_extrait.nF ).select(film_extrait.nF, film_extrait.titre, notes_extrait.nU, notes_extrait.note)
J.show(10000)

+---+--------------------+---+----+
| nF|               titre| nU|note|
+---+--------------------+---+----+
|  1|    Toy Story (1995)|457| 4.0|
|  1|    Toy Story (1995)|606| 4.0|
|  1|    Toy Story (1995)|869| 4.0|
|  1|    Toy Story (1995)| 75| 4.0|
|  1|    Toy Story (1995)|627| 3.5|
|  1|    Toy Story (1995)|682| 4.0|
|  1|    Toy Story (1995)|346| 4.0|
|  1|    Toy Story (1995)|845| 2.0|
|  1|    Toy Story (1995)|638| 5.0|
|  1|    Toy Story (1995)|196| 3.0|
|  1|    Toy Story (1995)| 44| 5.0|
|  1|    Toy Story (1995)|760| 5.0|
|  1|    Toy Story (1995)|392| 3.5|
|  1|    Toy Story (1995)|323| 4.0|
|  1|    Toy Story (1995)|545| 5.0|
|  1|    Toy Story (1995)|150| 4.0|
|  1|    Toy Story (1995)|667| 4.0|
|  1|    Toy Story (1995)|565| 5.0|
|  1|    Toy Story (1995)| 46| 4.5|
|  2|      Jumanji (1995)|796| 1.5|
|  2|      Jumanji (1995)|353| 3.0|
|  2|      Jumanji (1995)|390| 2.0|
|  2|      Jumanji (1995)|542| 3.0|
|  3|Grumpier Old Men ...|979| 3.0|
|  3|Grumpier Old Men ...| 8

# Jointure parallèle par Hachage et partitionnement



*   On a récupérer un extrait de films

*   On a partitionné cet extrait en 3 partition

*   On a créer 3 groupe de données dans chaques partitions (c'est notre fonction H )

*   On shuffle la table de films , c'est à dire qu'on join les groupes avec le même numéro ( c'est l'étape de répartition )








In [96]:
# les numéros de film (nF) sont compris entre 1 et 99

def Decouper_Film(iterateur):

  suivant = next(iterateur, None)
  groupes = [[] for x in range(3)]

  while suivant is not None:

    nf = suivant.nF
    if nf < 33:
      p=0
    elif nf < 66:
      p=1
    else:
      p=2

    groupes[p].append((p, *suivant))
    suivant = next(iterateur, None)

  for groupe in groupes:
    for film in groupe:
      yield film
      

In [97]:
names = ["p"] + film_extrait.schema.fieldNames()

F1= film_extrait.rdd.mapPartitions(Decouper_Film).toDF(names)
showPartitions(F1,50)

partition 0
+---+---+----------------------------------------------------+-------------------------------------------------+
|p  |nF |titre                                               |genres                                           |
+---+---+----------------------------------------------------+-------------------------------------------------+
|0  |9  |Sudden Death (1995)                                 |[Action]                                         |
|0  |4  |Waiting to Exhale (1995)                            |[Comedy, Drama, Romance]                         |
|0  |20 |Money Train (1995)                                  |[Action, Comedy, Crime, Drama, Thriller]         |
|0  |12 |Dracula: Dead and Loving It (1995)                  |[Comedy, Horror]                                 |
|0  |13 |Balto (1995)                                        |[Adventure, Animation, Children]                 |
|0  |25 |Leaving Las Vegas (1995)                            |[Drama, Romance]      



*   On shuffle les données, c'est à dire qu'on associe les données qui ont le même numéro de groupe ( le même H(films_extrait.nF) )



In [98]:
F2_ = F1.repartitionByRange(3, col('p'))

showPartitions(F2_,50)

partition 0
+---+---+---------------------------------------------------------------+-------------------------------------------------+
|p  |nF |titre                                                          |genres                                           |
+---+---+---------------------------------------------------------------+-------------------------------------------------+
|0  |9  |Sudden Death (1995)                                            |[Action]                                         |
|0  |4  |Waiting to Exhale (1995)                                       |[Comedy, Drama, Romance]                         |
|0  |20 |Money Train (1995)                                             |[Action, Comedy, Crime, Drama, Thriller]         |
|0  |12 |Dracula: Dead and Loving It (1995)                             |[Comedy, Horror]                                 |
|0  |13 |Balto (1995)                                                   |[Adventure, Animation, Children]               



*   F2 contient pour chaque numéro de partition , la liste de films associées (ça facilite la jointure sur le numéro de partition )



In [99]:
F2 = F2_.groupBy('p').agg( collect_list( array('nF','titre') ).alias('films') )
F2.show()

+---+--------------------+
|  p|               films|
+---+--------------------+
|  0|[[9, Sudden Death...|
|  1|[[45, To Die For ...|
|  2|[[94, Beautiful G...|
+---+--------------------+




*   On a récupéré un extrait de notes 

*   On l'a partitionné en 3 partitions

*   On crée 3 groupes de donneés dans chaque partitions ( fonction H)

*   On shuffle la table de notes , c'est à dire qu'on join les groupes avec le même numéro ( c'est l'étape de répartition )





In [100]:
# On découpe les données de la même façon qu'on a découpé les données film , ansi on a la même fonction H.
# De cette manière on est sûr que pour les deux tables ( notes et films), pour chaque partition on aura les films qui correspondent.

def Decouper_Notes(iterateur):

  suivant = next(iterateur, None)
  groupes = [[] for x in range(3)]

  while suivant is not None:

    nf = suivant.nF
    if nf < 33:
      p=0
    elif nf < 66:
      p=1
    else:
      p=2

    groupes[p].append((p, *suivant))
    suivant = next(iterateur, None)

  for groupe in groupes:
    for note in groupe:
      yield note


In [101]:
names = ["p"] + notes_extrait.schema.fieldNames()

N1 = notes_extrait.rdd.mapPartitions(Decouper_Notes).toDF(names)



*   On shuffle les données, c'est à dire qu'on associe les données qui ont le même numéro de groupe ( le même H(notes_extrait.nF) )





In [102]:
N2_ = N1.repartitionByRange(3, col('p'))
showPartitions(N2_,100)

partition 0
+---+---+---+----+-----+
|p  |nF |nU |note|annee|
+---+---+---+----+-----+
|0  |19 |76 |1.5 |2011 |
|0  |25 |832|5.0 |1998 |
|0  |5  |296|3.5 |2005 |
|0  |2  |796|1.5 |2015 |
|0  |2  |353|3.0 |2003 |
|0  |28 |815|5.0 |1996 |
|0  |5  |196|4.0 |1996 |
|0  |29 |882|5.0 |1996 |
|0  |12 |459|3.0 |2015 |
|0  |1  |457|4.0 |2017 |
|0  |1  |606|4.0 |2002 |
|0  |11 |236|4.0 |2001 |
|0  |10 |341|3.0 |1997 |
|0  |10 |481|2.0 |1996 |
|0  |1  |869|4.0 |2012 |
|0  |6  |733|4.5 |2006 |
|0  |6  |384|3.0 |2000 |
|0  |6  |703|4.0 |2003 |
|0  |11 |455|3.0 |1996 |
|0  |6  |153|4.5 |2004 |
|0  |10 |421|4.0 |1999 |
|0  |32 |160|4.0 |2000 |
|0  |32 |848|4.0 |2007 |
|0  |22 |66 |2.0 |2003 |
|0  |16 |12 |4.0 |1999 |
|0  |14 |157|3.0 |1996 |
|0  |14 |112|3.0 |1996 |
|0  |17 |523|5.0 |1998 |
|0  |21 |638|4.0 |1996 |
|0  |7  |611|2.0 |1999 |
|0  |21 |557|4.0 |2002 |
|0  |3  |979|3.0 |1997 |
|0  |7  |368|2.5 |2006 |
|0  |15 |875|4.0 |1996 |
|0  |29 |592|4.0 |2000 |
|0  |31 |767|3.0 |1996 |
|0  |25 |339|



*  N2 contient pour chaque numéro de partition , la liste de notes associées (ça facilite la jointure sur le numéro de partition )



In [103]:
# On définit une liste pour chaque partition

N2 = N2_.groupBy('p').agg( collect_list( array('nF','nU', 'note') ).alias('notes') )
N2.show()

+---+--------------------+
|  p|               notes|
+---+--------------------+
|  0|[[19.0, 76.0, 1.5...|
|  1|[[62.0, 691.0, 5....|
|  2|[[95.0, 400.0, 3....|
+---+--------------------+





*   On associé les partitions ayant le même numéro dans les tables de films (F2) et de notes (N2)


In [104]:
J1 = F2.join(N2, F2.p == N2.p )
J1.show()

+---+--------------------+---+--------------------+
|  p|               films|  p|               notes|
+---+--------------------+---+--------------------+
|  0|[[9, Sudden Death...|  0|[[19.0, 76.0, 1.5...|
|  1|[[45, To Die For ...|  1|[[62.0, 691.0, 5....|
|  2|[[94, Beautiful G...|  2|[[95.0, 400.0, 3....|
+---+--------------------+---+--------------------+





*   Jointure par tri fusion


In [105]:
def jointure_film_notes(iterateur):

  suivant = next(iterateur, None)  
  
  while suivant is not None:
  
    films = suivant.films
    notes = suivant.notes
    
    # TRI

    sorted_films = sorted(films, key=lambda t: int(t[0])) # Trie selon la clé de jointure "nF"
    sorted_notes = sorted(notes, key=lambda t: int(t[0])) # Trie selon la clé de jointure "nF"

    # FUSION

    for n in sorted_notes :
      for f in sorted_films :
        if( int(float(n[0])) == int(f[0]) ):        # si films.nF == notes.nF
          yield ( int(f[0]), f[1], int(float(n[1])) , n[2])
          break
 
  
    suivant = next(iterateur, None)  



In [106]:
names = ["nF","titre","nU","note"]

J2 = J1.rdd.mapPartitions(jointure_film_notes).toDF(names)
J2.show(10000)

+---+--------------------+---+----+
| nF|               titre| nU|note|
+---+--------------------+---+----+
|  1|    Toy Story (1995)|457| 4.0|
|  1|    Toy Story (1995)|606| 4.0|
|  1|    Toy Story (1995)|869| 4.0|
|  1|    Toy Story (1995)| 75| 4.0|
|  1|    Toy Story (1995)|627| 3.5|
|  1|    Toy Story (1995)|682| 4.0|
|  1|    Toy Story (1995)|346| 4.0|
|  1|    Toy Story (1995)|845| 2.0|
|  1|    Toy Story (1995)|638| 5.0|
|  1|    Toy Story (1995)|196| 3.0|
|  1|    Toy Story (1995)| 44| 5.0|
|  1|    Toy Story (1995)|760| 5.0|
|  1|    Toy Story (1995)|392| 3.5|
|  1|    Toy Story (1995)|323| 4.0|
|  1|    Toy Story (1995)|545| 5.0|
|  1|    Toy Story (1995)|150| 4.0|
|  1|    Toy Story (1995)|667| 4.0|
|  1|    Toy Story (1995)|565| 5.0|
|  1|    Toy Story (1995)| 46| 4.5|
|  2|      Jumanji (1995)|796| 1.5|
|  2|      Jumanji (1995)|353| 3.0|
|  2|      Jumanji (1995)|390| 2.0|
|  2|      Jumanji (1995)|542| 3.0|
|  3|Grumpier Old Men ...|979| 3.0|
|  3|Grumpier Old Men ...| 8

# Comparaison du contenu de J avec celui de J2



*   On vérifie que le contenue de J et de J2 soit identique. Pour cela il faut que leur schéma et leur contenue soient identiques.



In [107]:
J2.count(), J.count()

(185, 185)

In [108]:
def equal_DataFrame(df1,df2):
  return (df1.schema == df2.schema) and (df1.collect() == df2.collect())

In [109]:
equal_DataFrame(J,J2)

True

# Jointure par diffusion ( Broadcast join )



*   Le but est de répliquer et difuser les données



In [111]:

def numeroter_partition(partID,iterateur):

  suivant = next(iterateur, None)
  
  while suivant is not None:
    yield (partID, *suivant)
    suivant = next(iterateur, None)






*   On définit une liste de films pour chaque partition de la table de films



In [116]:
names = ["p_f"] + film_extrait.schema.fieldNames()

T1_ = film_extrait.rdd.mapPartitionsWithIndex(numeroter_partition).toDF(names)

T1 = T1_.groupBy('p_f').agg( collect_list( array('nF','titre') ).alias('films') )

T1.show()

+---+--------------------+
|p_f|               films|
+---+--------------------+
|  0|[[94, Beautiful G...|
|  1|[[14, Nixon (1995...|
|  2|[[72, Kicking and...|
+---+--------------------+





*   On définit une liste de notes pour chaque partition de la table de notes



In [117]:
names = ["p_n"] + notes_extrait.schema.fieldNames()

T2_ = notes_extrait.rdd.mapPartitionsWithIndex(numeroter_partition).toDF(names)

T2 = T2_.groupBy('p_n').agg( collect_list( array('nF','nU','note') ).alias('notes') )

T2.show()

+---+--------------------+
|p_n|               notes|
+---+--------------------+
|  0|[[19.0, 76.0, 1.5...|
|  1|[[15.0, 875.0, 4....|
|  2|[[19.0, 151.0, 2....|
+---+--------------------+





*   On envoie toutes les partitions de films sur chaque partition de notes ( en utilisant un cross join)



In [121]:
T = T2.crossJoin(T1)
T.show()

+---+--------------------+---+--------------------+
|p_n|               notes|p_f|               films|
+---+--------------------+---+--------------------+
|  0|[[19.0, 76.0, 1.5...|  0|[[94, Beautiful G...|
|  0|[[19.0, 76.0, 1.5...|  1|[[14, Nixon (1995...|
|  0|[[19.0, 76.0, 1.5...|  2|[[72, Kicking and...|
|  1|[[15.0, 875.0, 4....|  0|[[94, Beautiful G...|
|  1|[[15.0, 875.0, 4....|  1|[[14, Nixon (1995...|
|  1|[[15.0, 875.0, 4....|  2|[[72, Kicking and...|
|  2|[[19.0, 151.0, 2....|  0|[[94, Beautiful G...|
|  2|[[19.0, 151.0, 2....|  1|[[14, Nixon (1995...|
|  2|[[19.0, 151.0, 2....|  2|[[72, Kicking and...|
+---+--------------------+---+--------------------+





*   Pour chaque partitions de notes, on a une liste qui contient autant de liste  de films  qu'il y a de partitions dans la table de films



In [145]:
B = T.groupBy('p_n').agg( first(T.notes).alias('notes'), collect_list(T.films).alias('films') )

B.show()

+---+--------------------+--------------------+
|p_n|               notes|               films|
+---+--------------------+--------------------+
|  2|[[19.0, 151.0, 2....|[[[94, Beautiful ...|
|  0|[[19.0, 76.0, 1.5...|[[[94, Beautiful ...|
|  1|[[15.0, 875.0, 4....|[[[94, Beautiful ...|
+---+--------------------+--------------------+





*   On effectue les jointures : 
      - Pour chaque partition de note, on l'a join avec chacune des partions de films

*   On utilise un tri fusion pour les jointures




In [146]:
def jointure_local(iterateur):

  suivant = next(iterateur, None)  
  
  while suivant is not None:
  
    List_List_films = suivant.films # Toutes les partitions de films (= tous les films)
    List_notes = suivant.notes
    
    # TRI
    sorted_notes = sorted(List_notes, key=lambda t: int(t[0])) # Trie selon la clé de jointure "nF"

    # FUSION

    # Pour la partition de notes, on l'a joint avec toutes partitions de films

    for n in sorted_notes :
      for List_film in List_List_films : # On récupère la partition de films

        sorted_films = sorted(List_film, key=lambda t: int(t[0])) # Trie selon la clé de jointure "nF"

        for f in sorted_films :
          if( int(float(n[0])) == int(f[0]) ):        # si films.nF == notes.nF
            yield ( int(f[0]), f[1], int(float(n[1])) , n[2])
            break
 
  
    suivant = next(iterateur, None)  



In [147]:
names = ["nF","titre","nU","note"]

res = B.rdd.mapPartitions(jointure_local).toDF(names)

In [148]:
res.show(1000)

+---+--------------------+---+----+
| nF|               titre| nU|note|
+---+--------------------+---+----+
|  1|    Toy Story (1995)|392| 3.5|
|  1|    Toy Story (1995)|323| 4.0|
|  1|    Toy Story (1995)|545| 5.0|
|  1|    Toy Story (1995)|150| 4.0|
|  1|    Toy Story (1995)|667| 4.0|
|  1|    Toy Story (1995)|565| 5.0|
|  1|    Toy Story (1995)| 46| 4.5|
|  2|      Jumanji (1995)|542| 3.0|
|  5|Father of the Bri...|900| 3.0|
|  6|         Heat (1995)| 24| 4.0|
|  6|         Heat (1995)|413| 4.0|
|  6|         Heat (1995)|931| 4.0|
|  7|      Sabrina (1995)|839| 4.0|
| 10|    GoldenEye (1995)|533| 4.0|
| 10|    GoldenEye (1995)|875| 3.0|
| 11|American Presiden...|638| 3.0|
| 11|American Presiden...| 65| 3.0|
| 12|Dracula: Dead and...|390| 3.0|
| 14|        Nixon (1995)|850| 4.0|
| 17|Sense and Sensibi...|368| 2.0|
| 18|   Four Rooms (1995)|131| 4.0|
| 19|Ace Ventura: When...|151| 2.0|
| 21|   Get Shorty (1995)|151| 4.0|
| 25|Leaving Las Vegas...|576| 4.0|
| 25|Leaving Las Vegas...|72